# Predicting House Sale Prices

## Import Necessary Libraries

In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

##  Housing Dataset

Here is the housing data for the city of Ames, Iowa, United States from 2006 to 2010. 

In [351]:
df = pd.read_csv("AmesHousing.tsv", sep="\t")
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


## Feature Engineering, Data Exploration, and Data Cleaning

Steps:
1. See which of the features have missing values. For the features that has more than 5% of values missing, drop the features.
2. For the text features, drop those that have missing values.
3. For the numeric features that have up to 5% missing values, replace the missing values with the most occurring value (i.e mode).
4. Remove the columns that leak about the final sale and columns useless for building models.

In [359]:
def transform_features(df):
    
    feature_data = {"Feature": df.isnull().sum().keys().tolist(), "Missing Values": df.isnull().sum().values.tolist()}
    df_features = pd.DataFrame(feature_data)
    rows,cols = df.shape
    df_features['Percent Missing'] = 100*df_features['Missing Values']/rows
    features = df_features[df_features['Percent Missing'] > 5]['Feature'].tolist()
    df = df.drop(features, axis=1)
    
    text_feature_data = {"Feature": df.select_dtypes(include=['object']).isnull().sum().keys().tolist(), "Missing Values": df.select_dtypes(include=['object']).isnull().sum().values.tolist()}
    df_text_features = pd.DataFrame(text_feature_data)
    text_features = df_text_features[df_text_features['Missing Values'] > 0]['Feature'].tolist()
    df = df.drop(text_features, axis=1)
    
    numeric_dtypes_missing_values = df.select_dtypes(include=['int64','float64']).isnull().sum()
    numeric_features = numeric_dtypes_missing_values[numeric_dtypes_missing_values > 0]
    df = df.fillna(value=numeric_features)
    
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df = df.drop(['Year Built', 'Year Remod/Add'], axis=1)
    df = df.drop([1702,2180,2181], axis=0)
    df = df.drop(["Order","PID","Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

## Feature Selection

Steps:
1. Generate a correlation heatmap matrix of the numerical features in the training data set with respect to SalePrice.
2. Determine which numerical features should be converted to categorical features as their numbers don't have any semantic meaning. Aim for the features with few unique values with a significant majority (95%) of the values belonging to a specific category.

In [360]:
def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = transform_df.select_dtypes(include=['int64', 'float64'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

## Train and Test the Model

The function train_test can perform validation (k=0) or K-Fold cross validation (k > 0). 

In [361]:
def train_test(df, k=0):
    if k < 0:
        raise Exception("Value for k is negative!")
        
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k==0:
        row,col = df.shape
        train_test_cutoff = row*0.7
        train = numeric_df.iloc[:int(train_test_cutoff)]
        test = numeric_df.iloc[int(train_test_cutoff):]
        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
    
        return np.sqrt(mse)
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
        

## Complete Automated ML Pipeline

In [362]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_test(filtered_df, k=4)
rmse

[24344.031226139265, 29103.803299408657, 35693.48786672571, 26965.662593750727]


29026.74624650609